# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
city_df = pd.read_csv('weather_data.csv')
city_df.head()

,Unnamed: 0,City,Country,Latitude,Longitude,Humidity,Temperature,Cloudiness,Wind Speed
0,1,Rikitea,pf,-23.12,-134.97,78.0,70.05,0.0,0.00
1,2,Kapaa,us,22.08,-159.32,69.0,82.40,100.0,11.48
2,3,Dikson,ru,73.51,80.55,69.0,25.08,100.0,11.48
3,4,Bluff,nz,-23.58,149.07,11.0,100.40,90.0,9.17
4,5,Antalaha,mg,-14.90,50.28,91.0,70.48,90.0,9.17


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [10]:
#Create a heatmap for humidity in the cities
m = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(city_df[["Latitude", "Longitude"]], weights=city_df['Humidity'],
             max_intensity=100, point_radius=3, dissipating=False)

m.add_layer(heatmap_layer)

m

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
#Create a dataframe of cities with the ideal weather
ideal_df = city_df[(city_df['Temperature'] < 75) &
                   (city_df['Temperature'] > 60) &
                   (city_df['Wind Speed'] < 10) &
                   (city_df['Humidity'] < 40)].reset_index(drop=True)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
#Create a column for hotel name
ideal_df['Hotel Name'] = ""
ideal_df

,Unnamed: 0,City,Country,Latitude,Longitude,Humidity,Temperature,Cloudiness,Wind Speed,Hotel Name
0,87,Uruzgan,af,32.93,66.63,19.0,61.25,20.0,2.48,
1,129,Khorixas,na,-20.37,14.96,18.0,70.71,32.0,7.05,
2,184,Hami,cn,42.84,93.51,13.0,70.73,13.0,1.99,
3,236,Hukuntsi,bw,-23.98,21.76,17.0,66.39,0.0,4.47,
4,309,Tongliao,cn,43.61,122.27,15.0,60.38,100.0,4.70,
5,316,Jumla,np,29.28,82.18,38.0,64.49,1.0,4.70,
6,323,Opuwo,na,-18.06,13.84,20.0,72.71,17.0,1.01,
7,395,Awbari,ly,26.59,12.77,21.0,69.26,75.0,4.70,
8,421,Yumen,cn,40.29,97.04,25.0,62.78,90.0,6.29,
9,480,Sarkand,kz,45.41,79.91,20.0,74.57,100.0,9.04,


In [13]:
#Base url for api call
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Set up parameters dictionary for api call
params = {
    'rankby': 'distance',
    'keyword': 'hotel',
    'key': g_key
}

#Create loop to retrieve latitude and longitude data and run api call

for index, row in ideal_df.iterrows():
    lat = row['Latitude']
    lon = row['Longitude']
    
    #Add location to parameters
    
    params['location'] = f"{lat},{lon}"

# API request   
    hotel_json = requests.get(base_url, params=params).json()
    
#Adding hotel to dataframe

    if hotel_json['status'] == 'OK':
        ideal_df.loc[index, 'Hotel Name'] = hotel_json['results'][0]['name']
    else:
        pass

In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row

# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_df.iterrows()]
locations = ideal_df[["Latitude", "Longitude"]]

In [15]:
# Add marker layer ontop of heat map
#Create a map with markers for hotels
hotel_layer = gmaps.marker_layer(
    ideal_df[["Latitude", "Longitude"]],
    info_box_content=hotel_info
)

fig = gmaps.figure()
fig.add_layer(hotel_layer)
fig


# Display Map
fig = gmaps.figure()

fig.add_layer(heatmap_layer)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))